<a href="https://colab.research.google.com/github/thettt123/financial_math_project/blob/main/%EA%B8%B0%EB%A7%90%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=4e4238afb07f83259f96fd434929bc92b7b7adaea34ecb8fa2fbac39b832e9c6
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [50]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import feedparser


# 종목
stocks = {
    "SK하이닉스": "000660",
        "삼성전자": "005930",
        "LG에너지솔루션": "373220",
        "현대차": "005380",
        "카카오": "035720",
        "NAVER": "035420",
        "POSCO홀딩스": "005490",
        "기아": "000270",
        "셀트리온": "068270",
        "삼성SDI": "006400",
        "신한지주": "055550",
        "삼성바이오로직스": "207940",
        "현대모비스": "012330",
        "SK이노베이션": "096770",
        "삼성물산": "028260",
        "SK스퀘어": "402340",
        "LG화학": "051910",
        "LG전자": "066570",
        "KB금융": "105560",
        "카카오뱅크": "323410",
        "하나금융지주": "086790",
        "삼성에스디에스": "018260",
        "크래프톤": "259960",
        "우리금융지주": "316140",
        "한국전력": "015760",
        "KT&G": "033780",
        "SK텔레콤": "017670",
        "엔씨소프트": "036570",
        "고려아연": "010130",
        "삼성전기": "009150",
        "에코프로비엠": "247540"
}

# 가격 가져오기
def get_price_from_html(code):
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    try:
        # KRX 가격
        krx_section = soup.select_one("#rate_info_krx .no_today em")
        krx_span = krx_section.find("span", class_="blind")
        krx = int(krx_span.text.replace(',', '')) if krx_span else None

        # NXT 가격
        nxt_section = soup.select_one("#rate_info_nxt .no_today em span.blind")
        nxt = int(nxt_section.text.replace(',', '')) if nxt_section else None

        return krx, nxt
    except Exception as e:
        print(f"[ERROR] {code} - {e}")
        return None, None

# 정보 가져오기
def get_naver_news_rss(stock_name):
    query = urllib.parse.quote(f"{stock_name} 주식 site:naver.com")
    rss_url = f"https://news.google.com/rss/search?q={query}&hl=ko&gl=KR&ceid=KR:ko"
    feed = feedparser.parse(rss_url)

    articles = []
    for entry in feed.entries[:5]:
        title = entry.title
        link = entry.link
        articles.append((title, link))
    return articles

# 출력 테스트 (차익률 0.5% 이상만 출력)
for name, code in stocks.items():
    krx_price, nxt_price = get_price_from_html(code)
    if krx_price and nxt_price:
        spread = abs(krx_price - nxt_price)
        percent = spread / krx_price * 100
        if percent >= 0.5: # 차익률
            print(f"{name}: KRX={krx_price}, NXT={nxt_price}, 차익={spread}원, 차익률={percent:.2f}%")
            news = get_naver_news_rss(name)
            for title, link in news:
                print(f"    - {title} ({link})")
    else:
        print(f"[오류] {name}: 가격 정보를 가져올 수 없습니다.")



SK하이닉스: KRX=257000, NXT=259000, 차익=2000원, 차익률=0.78%
    - [맥쿼리] SK하이닉스 (000660.KS) - 네이버 프리미엄콘텐츠 (https://news.google.com/rss/articles/CBMif0FVX3lxTE16WV9ZNVBDMVlmVGFVXy03MUlpVVdDUDlGa18zZTJ5bFVNemFHRGtJeXhRUWZRNWlMNm1uWXR0WXFGQVBRbjdhaDJFVUpWekw0dFpTczNBM0sxVzNFUFBpRG5MRDh5R3ZELUt3SzNBbVhNNFVUX3dDX3NBdk9WeGs?oc=5)
    - SK하이닉스 주가, 모건스탠리 보고서와 정치 논란으로 횡보 : 네이버 블로그 - NAVER (https://news.google.com/rss/articles/CBMiW0FVX3lxTFA5OU1QSFBxdTFMU1A2ZUNPYlgyYVFrdHNpZjB6RkE4Q0FSUkNVX0Rjd3RyR1psV0poTWFseXY5R0Vib3piVURlc1dSS0ExbzQ2RUFJUXlqRmZONGs?oc=5)
    - SK하이닉스 35주 분할매도하여 수익실현!(수익 2,282,322원) : 네이버 블로그 - NAVER (https://news.google.com/rss/articles/CBMiggFBVV95cUxQYjZEQW9UeEt3SVZZakEtRFhKOGEtdVQ1RUVFdkd2ZzlpWGd1eFptWVFBMGFuZllmaWwzT2MtZDZSZWZ1ZHhFZkhpQkpJSGw1UEV5WERNUHhfdkhDcWxLUDBTOGVNM0N4NlgwNlQtamRZMkhzMUkzcnpoSU0tOG1kb3BB?oc=5)
    - SK하이닉스 신고가 달성, 신고가 쓸 종목이 널렸는데 국장을 아직도 안 해요? [주식 뭐사지?] - 네이버 프리미엄콘텐츠 (https://news.google.com/rss/articles/CBMiiAFBVV95cUxOTFJhSmZaUlh6eGFnUDRwU1JZcktOZzhjVnNWLT